# Week 4 Demo

This demo is created for **MLAB's meDecal**. It is inspired by the manuscript, **A Primer on Deep Learning in Genomics** (*Nature Genetics, 2018*)

## Python Libraries
As usual, we will first import the tools we need. 

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
import torch

## 0. Background

In this demo, we will use CNN to approach an important problem in functional genomics: **the discovery of transcription-factor binding sites in DNA**.

But wait.. what is transcription-factor binding sites? check out this [deck](https://docs.google.com/presentation/d/1FpBBlRK-rN4BDHJslq1kbJ_t0TILcY9RRiZvv5RTjmI/edit?usp=sharing) for more background on the Biology.






As we go through this notebook, we will  design a neural network that can discover binding motifs in DNA based on the results of an assay that determines whether a longer DNA sequence binds to the transcription factor (protein) or not. Here, the longer DNA sequences are our *independent variables* (or *predictors*), while the positive or negative response of the assay is the *dependent variable* (or *response*).

We will use simulated data that consists of DNA sequences of length 50 bases (chosen to be artificially short so that the data is easy to play around with), and is labeled with 0 or 1 depending on the result of the assay. Our goal is to build a classifier that can predict whether a particular sequence will bind to the protein.

(Spoiler alert: the true regulatory motif is *`CGACCGAACTCC`*. Of course, the neural network doesn't know this.)

## 1. Curate the Data

![alt text](https://github.com/athenaleong/Medecal_Misc/blob/main/step_1.png?raw=true)

In order to train the neural network, we must load and preprocess the data, which consists of DNA sequences and their corresponding labels.By processing this data, the network will learn to distinguish sequences that bind to the transcription factor from those that do not. 

We start by loading the simulated data from an external repository.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests

SEQUENCES_URL = 'https://raw.githubusercontent.com/abidlabs/deep-learning-genomics-primer/master/sequences.txt'

sequences = requests.get(SEQUENCES_URL).text.split('\n')
sequences = list(filter(None, sequences))  # This removes empty sequences.

# Let's print the first few sequences.
pd.DataFrame(sequences, index=np.arange(1, len(sequences)+1), 
             columns=['Sequences']).head()

The next  step is to organize the data into a format that can be passed into a deep learning algorithm. Most deep learning algorithms accept data in the form of vectors or matrices (or more generally, tensors). 

To get each DNA sequence in the form of a matrix, we use _one-hot encoding_, which encodes every base in a sequence in the form of a 4-dimensional vector, with a separate dimension for each base. We place a "1" in the dimension corresponding to the base found in the DNA sequence, and "0"s in all other slots. We then concatenate these 4-dimensional vectors together along the bases in the sequence to form a matrix. 

In the cell below, we one-hot encode the simulated DNA sequences, and show an example of what the one-hot encoded sequence looks like:

In [ ]:
# The LabelEncoder encodes a sequence of bases as a sequence of integers.
integer_encoder = LabelEncoder()  
# The OneHotEncoder converts an array of integers to a sparse matrix where 
# each row corresponds to one possible value of each feature.
one_hot_encoder = OneHotEncoder(categories='auto')   
input_features = []

for sequence in sequences:
  integer_encoded = integer_encoder.fit_transform(list(sequence))
  integer_encoded = np.array(integer_encoded).reshape(-1, 1)
  one_hot_encoded = one_hot_encoder.fit_transform(integer_encoded)
  input_features.append(one_hot_encoded.T.toarray())

np.set_printoptions(threshold=40)
input_features = np.stack(input_features)
print("Example sequence\n-----------------------")
print('DNA Sequence #1:\n',sequences[0][:10],'...',sequences[0][-10:])
print('One hot encoding of Sequence #1:\n',input_features[0])

Similarly, we can go ahead and load the labels. In this case, the labels are structured as follows: a "1" indicates that a protein bound to the sequence, while a "0" indicates that the protein did not. While we could use the labels as a vector, it is often easier to similarly one-hot encode the labels, as we did the features. We carry out that here:

In [ ]:
LABELS_URL = 'https://raw.githubusercontent.com/abidlabs/deep-learning-genomics-primer/master/labels.txt'

labels = requests.get(LABELS_URL).text.split('\n')
labels = list(filter(None, labels))  # removes empty sequences

one_hot_encoder = OneHotEncoder(categories='auto')
labels = np.array(labels).reshape(-1, 1)
input_labels = one_hot_encoder.fit_transform(labels).toarray()

print('Labels:\n',labels.T)
print('One-hot encoded labels:\n',input_labels.T)

Next, We will go ahead and split the data into three different sub-datasets:

(1) Training dataset: a dataset used to fit the parameters of a model or to define the weights of connections between neurons of a neural network.

(2) Validation dataset: a second dataset used to minimize overfitting. The weights of the network are not adjusted with this data set. After each training cycle, if the accuracy over the training data set increases, but the accuracy over the validation data set stays the same or decreases, then there is overfitting on the neural network.

(3) Testing dataset: is a third dataset not included in the training nor validation data sets. After all the training and validation cycles are complete, this dataset is used only for testing the final solution in order to measure the actual predictive power of the neural network on new examples.

-----------

In [ ]:
from sklearn.model_selection import train_test_split
import torch


X_train_val, X_test, y_train_val, y_test = train_test_split(
    input_features, input_labels, test_size=0.25, random_state=42)

X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.25)


X_train = X_train.astype(np.float32)
X_train = torch.from_numpy(X_train)
y_train = y_train.astype(int)
y_train = torch.from_numpy(y_train)

X_val = X_val.astype(np.float32)
X_val = torch.from_numpy(X_val)
y_val = y_val.astype(int)
y_val = torch.from_numpy(y_val)

X_test = X_test.astype(np.float32)
X_test = torch.from_numpy(X_test)
y_test = y_test.astype(int)
y_test = torch.from_numpy(y_test)


## 2. Select the Architecture and Train

![alt text](https://github.com/athenaleong/Medecal_Misc/blob/main/step_2.png?raw=true)

Next, we choose a neural network architecture to train the model. In this tutorial, we choose a simple 1D convolutional neural network (CNN), which is commonly used in deep learning for functional genomics applications.

A CNN learns to recognize patterns that are generally invariant across space, by trying to match the input sequence to a number of learnable "filters" of a fixed size. In our dataset, the filters will be motifs within the DNA sequences. The CNN may then learn to combine these filters to recognize a larger structure (e.g. the presence or absence of a transcription factor binding site). 

To construct the CNN, We will use the deep learning library `Pytorch`. 

- _Conv1D_: We define our convolutional layer to have 32 filters of size 12 bases.

- _MaxPooling_: After the convolution, we use a pooling layer to down-sample the output of the each of the 32 convolutional filters. Though not always required, this is a typical form of non-linear down-sampling used in CNNs.

- _Flatten_: This layer flattens the output of the max pooling layer, combining the results of the convolution and pooling layers across all 32 filters. 

- _Dense_ / _Linear_: The first Dense tensor creates a layer (dense_1) that compresses the representation of the flattened layer, resulting in smaller layer with 16 tensors, and the second Dense function converges the tensors into the output layer (dense_2) that consists of the two possible response values (0 or 1).

We can see the details of the architecture of the neural network we have created by running `print(model)`, which prints the dimensionality and number of parameters for each layer in our network. 

In [ ]:
import torch
import torch.nn as nn
from torch.optim import Adam
from torch.autograd import Variable

class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        
        #Set up layers
        self.conv1 = nn.Sequential(nn.Conv1d(4, 32, kernel_size=12),
                    nn.MaxPool1d(4),
                    nn.Flatten())
        self.fc1 = nn.Sequential(nn.Flatten(), nn.Linear(288, 16),
                                nn.ReLU())
        self.fc2 = nn.Sequential(nn.Linear(16,2), 
                                nn.Softmax())
        
        #Initialize optimizer
        self.optimizer = Adam(self.parameters())
        
        #Initialize loss function
        self.loss = nn.BCELoss()

    def forward(self, X):
        X = self.conv1(X)
        X = self.fc1(X)
        output = self.fc2(X)
        
        return output

In [ ]:
#Check out model summary
model = ConvNet()
print(model)

In [ ]:
def accuracy(prediction, target):
    """
    Helper function to calculate accuracy
    """
    correct = 0
    prediction = list(prediction.detach().numpy())
    for t, p in zip(prediction, target):
        
        if np.argmax(t) == np.argmax(p):
            correct += 1
    
    return correct / len(target)
    

def train_and_display(model, epoch):
    train_losses = []
    val_losses = []
    train_accuracies = []
    val_accuracies = []
    
    for e in range(epoch):
        
        #Compute training
        pred_train = model(X_train)
        pred_val = model(X_val)
        
        #Compute validation loss
        loss_train = model.loss(pred_train, y_train.float())
        loss_val = model.loss(pred_val, y_val.float())
        
        #Compute accuracy
        train_accuracy = accuracy(pred_train, y_train)
        val_accuracy = accuracy(pred_val, y_val)
        
        #Zero parameter gradient & Optimize loss
        model.optimizer.zero_grad()
        loss_train.backward()
        model.optimizer.step()
        
        train_losses.append(loss_train)
        val_losses.append(loss_val)
        train_accuracies.append(train_accuracy)
        val_accuracies.append(val_accuracy)
        
        #Logging at regular intervals
        if e % 5 == 0 or e == epoch - 1:
            print('Epoch : ',e+1, '\t', 'loss :', loss_val, '\t', 'accuracy:', val_accuracy)
            
            
    #Plot training metrics over time 
    f, (plt1, plt2) = plt.subplots(2)
    plt1.plot(train_losses, label='Training loss')
    plt1.plot(val_losses, label='Validation loss')
    plt1.set_title('model loss')
    plt1.set_ylabel('loss')
    plt1.set_xlabel('epoch')
    plt1.legend()
    plt2.plot(train_accuracies, label='Training accuracy')
    plt2.plot(val_accuracies, label='Validation accuracy')
    plt2.set_title('model accuracy')
    plt2.set_ylabel('accuracy')
    plt2.set_xlabel('epoch')
    plt2.legend()
    
    f.subplots_adjust(hspace=0.5)
    plt.show()
    
    

In [ ]:
model = ConvNet()
train_and_display(model, 100)


Similarly, we can plot the accuracy of our neural network on the binary classification task. The metric used in this example is the _binary accuracy_, which calculates the proportion of predictions that match labels or response variables. Other metrics may be used in different tasks -- for example, the _mean squared error_ is typically used to measure the accuracy for continuous response variables (e.g. polygenic risk scores, total serum cholesterol level, height, weight and systolic blood pressure).

## 3. Evaluate

![alt text](https://github.com/athenaleong/Medecal_Misc/blob/main/step_3.png?raw=true)

The best way to evaluate whether the network has learned to classify sequences is to evaluate its performance on a fresh test set consisting of data that it has not observed at all during training. Here, we evaluate the model on the test set and plot the results as a confusion matrix. Nearly every test sequence should be correctly classified.

In [ ]:
from sklearn.metrics import confusion_matrix
import itertools

y_predicted = model(X_test)
y_predicted = list(y_predicted.detach().numpy())

cm = confusion_matrix(np.argmax(y_test, axis=1), np.argmax(y_predicted, axis=1))

#Visualisation of confusion matrix
print('Confusion matrix:\n',cm)
cm = cm.astype('float') / cm.sum(axis = 1)[:, np.newaxis]
plt.imshow(cm, cmap=plt.cm.Blues)
plt.title('Normalized confusion matrix')
plt.colorbar()
plt.xlabel('True label')
plt.ylabel('Predicted label')
plt.xticks([0, 1]); plt.yticks([0, 1])
plt.grid(False)
for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
    plt.text(j, i, format(cm[i, j], '.2f'),
             horizontalalignment='center',
             color='white' if cm[i, j] > 0.5 else 'black')
